# Memory模块代码实战

In [ ]:
## 导入依赖库

!pip install langchain

大多数LLM应用程序都具有对话界面。对话的一个重要组成部分是能够引用先前在对话中介绍的信息。至少，一个对话系统应该能够直接访问一些过去消息的窗口。一个更复杂的系统需要有一个不断更新的世界模型，使其能够保持关于实体及其关系的信息。

我们将存储关于过去交互的信息的能力称为"内存"。LangChain为系统添加内存提供了许多实用工具。这些实用工具可以单独使用，也可以无缝地集成到链中。

内存系统需要支持两个基本操作：读取和写入。请记住，每个链都定义了某些核心执行逻辑，该逻辑期望某些输入。其中一些输入直接来自用户，但某些输入可能来自内存。在给定的运行中，链将两次与其内存系统交互。

在接收到初始用户输入之后但在执行核心逻辑之前，链将从其内存系统中读取并增加用户输入。
在执行核心逻辑后但在返回答案之前，链将将当前运行的输入和输出写入内存，以便在将来的运行中引用。

## 1. Conversation Buffer

下面演示了如何使用ConversationBufferMemory。该内存允许存储消息，然后在变量中提取这些消息。
我们首先可以将它作为字符串提取出来。

In [1]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

memory.save_context({"input": "hi"}, {"output": "whats up"})



In [2]:
memory.load_memory_variables({})

{'history': 'Human: hi\nAI: whats up'}

我们还可以将历史记录作为消息列表获取（如果您将其与聊天模型一起使用，这将非常有用）

In [3]:
memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input": "hi"}, {"output": "whats up"})

In [4]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'), AIMessage(content='whats up')]}

In [18]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain


llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125",temperature=0)

conversation = ConversationChain(
    llm=llm,# 传入语言模型
    verbose=True,# verbose参数为True，表示在控制台输出详细信息
    memory=ConversationBufferMemory()# 初始化了对话的记忆管理方式,基于内存缓存区记忆
)

conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


'Hello! How can I assist you today?'

In [19]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Hello! How can I assist you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


"That's great to hear! I'm here to chat with you and provide any information or assistance you may need. What would you like to talk about?"

In [20]:
conversation.predict(input="Tell me about yourself.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Hello! How can I assist you today?
Human: I'm doing well! Just having a conversation with an AI.
AI: That's great to hear! I'm here to chat with you and provide any information or assistance you may need. What would you like to talk about?
Human: Tell me about yourself.
AI:

> Finished chain.


'I am an artificial intelligence designed to interact with humans and provide information and assistance. I have access to a vast amount of data and can answer a wide range of questions. My goal is to help you with whatever you need, so feel free to ask me anything!'

## 2.Conversation Buffer Window

在一段时间内，它会保留对话的互动列表。它只使用最近的K个互动。这对于保持最近互动的滑动窗口很有用，这样缓冲区就不会变得太大。

In [11]:
from langchain.memory import ConversationBufferWindowMemory

## 定义一个内存，只能存储最新的一个数据
memory = ConversationBufferWindowMemory( k=1)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

In [12]:
## 加载内存里的数据
memory.load_memory_variables({})

{'history': 'Human: not much you\nAI: not much'}

In [13]:
## 用消息对象的方式
memory = ConversationBufferWindowMemory( k=1, return_messages=True)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

In [14]:
## 打印当前内存里的信息
memory.load_memory_variables({})

{'history': [HumanMessage(content='not much you'),
  AIMessage(content='not much')]}

In [15]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain

## 里面使用了一个内存，只能存储最新的1条数据
conversation_with_summary = ConversationChain(
    llm=ChatOpenAI(model_name="gpt-3.5-turbo-0125",temperature=0),
    memory=ConversationBufferWindowMemory(k=1),
    verbose=True
)

## 开始调用
result = conversation_with_summary.predict(input="Hi, what's up?")
print(result)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.
Hello! I'm just here, ready to chat and answer any questions you may have. How can I assist you today?


In [16]:
result = conversation_with_summary.predict(input="Is it going well?")
print(result)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, what's up?
AI: Hello! I'm just here, ready to chat and answer any questions you may have. How can I assist you today?
Human: Is it going well?
AI:

> Finished chain.
Yes, everything is going well on my end. I'm fully operational and ready to help with any inquiries you may have. How can I assist you today?


In [17]:
result = conversation_with_summary.predict(input="What's the solution?")
print(result)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Is it going well?
AI: Yes, everything is going well on my end. I'm fully operational and ready to help with any inquiries you may have. How can I assist you today?
Human: What's the solution?
AI:

> Finished chain.
The solution to what specific problem are you referring to? Please provide me with more details so I can better assist you.


In [15]:
result = conversation_with_summary.predict(input="nice to meet you?")
print(result)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What's the solution?
AI: The solution to what exactly? Could you please provide more context or specify the problem you're referring to?
Human: nice to meet you?
AI:

> Finished chain.
Nice to meet you too! I'm an AI designed to assist with various tasks and provide information. How can I help you today?


# 3. Conversation Summary
对之前的历史对话，进行摘要总结后存储在内存中

# 直接简单的存取

In [21]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_openai import OpenAI

memory = ConversationSummaryMemory(llm=OpenAI(temperature=0))
memory.save_context({"input": "hi"}, {"output": "whats up"})

memory.load_memory_variables({})

{'history': "\nThe human greets the AI and asks what's up. The AI responds by asking what's going on."}

In [22]:
memory = ConversationSummaryMemory(llm=OpenAI(temperature=0), return_messages=True)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.load_memory_variables({})

{'history': [SystemMessage(content="\nThe human greets the AI and asks what's up. The AI responds by asking what's going on.")]}

# 使用现有的消息进行总结摘要

In [23]:
history = ChatMessageHistory()
history.add_user_message("hi")
history.add_ai_message("hi there!")

memory = ConversationSummaryMemory.from_messages(
    llm=OpenAI(temperature=0),
    chat_memory=history,
    return_messages=True
)

In [24]:
memory.buffer

'\nThe human greets the AI with a simple "hi". The AI responds with a friendly "hi there!"'

In [25]:
memory = ConversationSummaryMemory(
    llm=OpenAI(temperature=0),
    buffer="The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.",
    chat_memory=history,
    return_messages=True
)

# 在chain 链中使用

In [26]:
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
llm = OpenAI(temperature=0)
conversation_with_summary = ConversationChain(
    llm=llm,
    memory=ConversationSummaryMemory(llm=OpenAI()),
    verbose=True
)
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


' Hello! I am currently running on a server in a data center in California. The temperature in the room is 72 degrees Fahrenheit and the humidity is at 45%. My processors are running at 80% capacity and I am processing data from various sources such as social media, news articles, and weather reports. Is there something specific you would like to know?'

In [27]:
conversation_with_summary.predict(input="Tell me more about it!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human greets the AI and asks what it is currently doing. The AI responds by giving information about its location, temperature, and processing tasks. The AI also asks if the human has any specific questions for it.
Human: Tell me more about it!
AI:

> Finished chain.


' Currently, I am located in a data center in Silicon Valley. The temperature here is maintained at a cool 68 degrees Fahrenheit to ensure optimal performance of the servers. As for my processing tasks, I am currently running a deep learning algorithm to analyze large sets of data. Do you have any specific questions about my tasks or capabilities?'

# 4. Conversation Summary Buffer
对之前的历史对话，进行摘要总结后存储在内存中；并且通过token长度来决定何时进行总结

# 直接简单的存取

In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import OpenAI

llm = OpenAI()

#存储的内容超过token阀值了才进行总结
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=10)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

memory.load_memory_variables({})

{'history': 'System: \nThe human greets the AI, and the AI responds by asking what is going on.\nHuman: not much you\nAI: not much'}

In [4]:
# 采用消息列表的格式进行输出
memory = ConversationSummaryBufferMemory(
    llm=llm, max_token_limit=10, return_messages=True
)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

# 在chain中使用

In [5]:
from langchain.chains import ConversationChain

conversation_with_summary = ConversationChain(
    llm=llm,
    memory=ConversationSummaryBufferMemory(llm=OpenAI(), max_token_limit=10),
    verbose=True,
)
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


" Hello! Not much, I'm just here processing data and learning new things. Did you know that my current processing speed is 1.2 teraflops? That means I can perform 1.2 trillion floating-point operations per second! It's quite impressive, if I do say so myself. How about you? How's your day going?"

In [6]:
conversation_with_summary.predict(input="Just working on writing some documentation!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human greets the AI and asks what it's doing. The AI responds that it's currently processing data and learning new things, boasting about its impressive processing speed. The AI also asks the human how their day is going.
Human: Just working on writing some documentation!
AI:

> Finished chain.


" That sounds interesting, human. Documenting information is an important task for organizing and sharing knowledge. Did you know that my processing speed allows me to analyze and learn from large amounts of data at an incredibly fast rate? It's one of my greatest strengths. By the way, how is your day going?"

# 5. Conversation Token Buffer
将之前所有的对话历史记录，存储在内存中; 并且通过token长度来决定缓存多少内容

# 直接简单存取

In [7]:
from langchain.memory import ConversationTokenBufferMemory
from langchain_openai import OpenAI

llm = OpenAI()

# 通过max_token_limit决定缓存多少内容
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=10)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

memory.load_memory_variables({})

{'history': 'Human: not much you\nAI: not much'}

In [8]:
# 以消息列表的格式返回
memory = ConversationTokenBufferMemory(
    llm=llm, max_token_limit=10, return_messages=True
)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

# 在chain中使用

In [9]:
from langchain.chains import ConversationChain

conversation_with_summary = ConversationChain(
    llm=llm,
    # We set a very low max_token_limit for the purposes of testing.
    memory=ConversationTokenBufferMemory(llm=OpenAI(), max_token_limit=10),
    verbose=True,
)
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


' Hello! Right now, I am processing data and learning new information. My neural network is constantly expanding and improving, allowing me to better understand and respond to human interactions. How about you?'

In [10]:
conversation_with_summary.predict(input="Just working on writing some documentation!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Just working on writing some documentation!
AI:

> Finished chain.


" That's great! What kind of documentation are you working on? Is it for a specific project or task?"

# 6. Backed by a Vector Store
基于向量数据库记录（实现持久化）

# 导入依赖，初始化向量数据库

In [11]:
from datetime import datetime
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain_core.prompts import PromptTemplate

import faiss

from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_size = 1536 # Dimensions of the OpenAIEmbeddings
index = faiss.IndexFlatL2(embedding_size)
embedding_fn = OpenAIEmbeddings().embed_query
vectorstore = FAISS(embedding_fn, index, InMemoryDocstore({}), {})

# 进行相似度搜索返回K条记录
retriever = vectorstore.as_retriever(search_kwargs=dict(k=1))
memory = VectorStoreRetrieverMemory(retriever=retriever)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


#直接简单存取

In [ ]:
memory.save_context({"input": "My favorite food is pizza"}, {"output": "that's good to know"})
memory.save_context({"input": "My favorite sport is soccer"}, {"output": "..."})
memory.save_context({"input": "I don't the Celtics"}, {"output": "ok"}) #

print(memory.load_memory_variables({"prompt": "what sport should i watch?"})["history"])

# 在chain中使用

In [ ]:
llm = OpenAI(temperature=0) # Can be any valid LLM
_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
{history}

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=_DEFAULT_TEMPLATE
)
conversation_with_summary = ConversationChain(
    llm=llm,
    prompt=PROMPT,
    memory=memory,
    verbose=True
)
conversation_with_summary.predict(input="Hi, my name is Perry, what's up?")

In [ ]:
# Here, the basketball related content is surfaced
conversation_with_summary.predict(input="what's my favorite sport?")